There we import tensorflow pre-trained models. Inception v4 and v2 are also available if needed

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



There we establish session for the whole notebook. On what is session and why tf.* code is not executed when you expect it to be read this article:

https://www.easy-tensorflow.com/tf-tutorials/basics/graph-and-session

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)
sess = tf.InteractiveSession()

There we declare tf.Variable for our adversarial image. This type is used for all dynamically changing obects (such as weights of the model). Note that tf.Variable actually does not store the value inside it, rather it stores *operations* allowing to get this value. Remember that the size of the tf.variable cannot be changed dinamically in the code. One can change its dimensionality, though. More about tf.Variables and the difference between them and tf.placeholders can be found here:

https://www.easy-tensorflow.com/tf-tutorials/basics/graph-and-session

In [37]:
image = tf.Variable(tf.zeros((299, 299, 3)))

tf.expand_dims(tf.Variable, axis=?) is equivalent to np.reshape(np.ndarray), but only adds one dimesion and numeration of axis is from the left to the right.  

In [ ]:
def inception(image, reuse):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v3_arg_scope(weight_decay=0.0)
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v3(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
    return logits, probs

logits, probs = inception(image, reuse=False)

In [36]:
alpha = tf.Tensor([[1., 2.], [1., 2,]])
print(alpha)

TypeError: __init__() missing 2 required positional arguments: 'value_index' and 'dtype'